In [4]:
import logfire

# Remember to set LOGFIRE_TOKEN environment variable

logfire.configure()
logfire.instrument_pydantic()

Logfire project URL: 
]8;id=375991;https://logfire.pydantic.dev/cshorten/weaviate-integration-testing\https://logfire.pydantic.dev/cshorten/weaviate-integration-testing]8;;\


In [5]:
from pydantic_ai import Agent

agent = Agent(  
    'openai:gpt-4o',
    system_prompt='Be concise, reply with one sentence.',  
)

result = await agent.run('How many feet are in a mile?')  
print(result.data)

22:36:46.040 agent run prompt=How many feet are in a mile?
22:36:46.041   preparing model and tools run_step=1
22:36:46.041   model request
22:36:48.152     Pydantic nullable validate_python
22:36:48.154     Pydantic nullable validate_python
22:36:48.154     Pydantic nullable validate_python
22:36:48.157     Pydantic nullable validate_python
22:36:48.726   handle model response
There are 5,280 feet in a mile.


In [7]:
import weaviate
import os
print("Connecting to Weaviate...")

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
    headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")},
)
print("Successfully connected to Weaviate...")

01:26:25.574 Pydantic _VectorizerConfigCreate validate_python
01:26:25.588 Pydantic _VectorizerConfigCreate validate_python
Connecting to Weaviate...
01:26:25.639 Pydantic ProtocolParams validate_python
01:26:25.640 Pydantic ProtocolParams validate_python
01:26:25.640 Pydantic ConnectionParams validate_python
01:26:25.641 Pydantic AdditionalConfig validate_python
01:26:25.641   Pydantic Timeout validate_python
Successfully connected to Weaviate...


In [10]:
courses_collection = weaviate_client.collections.get("Courses")

In [9]:
def search_weaviate_collection(weaviate_client, weaviate_collection, query):
    response = weaviate_collection.query.hybrid(query, limit=5)
    
    stringified_response = ""
    for idx, o in enumerate(response.objects):
        stringified_response += f"Search Result: {idx+1}:\n"
        for prop in o.properties:
            stringified_response += f"{prop}:{o.properties[prop]}"
        stringified_response += "\n"
    
    return stringified_response

In [11]:
search_weaviate_collection(weaviate_client, courses_collection, "Neural Networks")

'Search Result: 1:\ncourseDescription:Deep dive into neural networks, reinforcement learning, and deep learning architectures. Students will implement cutting-edge ML models and understand their theoretical foundations.courseDuration:48.0currentlyEnrolling:TruecourseTitle:Advanced Machine Learning\nSearch Result: 2:\ncourseDescription:Introduction to quantum mechanics, quantum circuits, and quantum algorithms. Covers basic principles of superposition, entanglement, and quantum gates.courseDuration:36.0currentlyEnrolling:FalsecourseTitle:Quantum Computing Fundamentals\n'

In [19]:
import nest_asyncio
import logfire
import asyncio
from dataclasses import dataclass
from typing import List

from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

# Configure logfire
logfire.configure()

@dataclass
class SearchContext:
    """Dependencies (context) for the search."""
    weaviate_client: any
    collection: any


class SearchQuery(BaseModel):
    """The search query and parameters."""
    query_text: str = Field(description="The search query to execute")
    max_results: int = Field(default=5, description="Maximum number of results to return")


class SearchResult(BaseModel):
    """Structured output from the AI."""
    summary: str = Field(description="A concise summary of the search results")
    relevant_courses: List[str] = Field(description="List of relevant course titles")
    recommendation: str = Field(description="A recommendation based on the search results")


@logfire.instrument("Initialize query analyzer agent")
def create_query_analyzer():
    return Agent(
        model="openai:gpt-4",
        deps_type=SearchContext,
        result_type=SearchQuery,
        system_prompt=(
            "You are a search query analyzer. "
            "Given a user's search request, format it into an appropriate query "
            "that will yield relevant course results."
        )
    )

query_analyzer = create_query_analyzer()


@query_analyzer.system_prompt
async def analyzer_system_prompt(ctx: RunContext[SearchContext]) -> str:
    with logfire.span('Generate analyzer system prompt'):
        return (
            "Analyze the user's search request and format it into a search query "
            "that will find relevant courses in the collection."
        )


@logfire.instrument("Initialize results summarizer agent")
def create_results_summarizer():
    return Agent(
        model="openai:gpt-4",
        deps_type=SearchContext,
        result_type=SearchResult,
        system_prompt=(
            "You are a course search assistant. "
            "Analyze the search results and provide a concise summary, "
            "list relevant courses, and make a recommendation."
        )
    )

results_summarizer = create_results_summarizer()


async def main():
    with logfire.span('Course search workflow') as workflow_span:
        try:
            workflow_span.set_attribute('search_type', 'course_search')
            
            deps = SearchContext(
                weaviate_client=weaviate_client,
                collection=courses_collection
            )

            user_request = "Neural Network courses"
            workflow_span.set_attribute('user_request', user_request)
            
            # Query analysis phase
            with logfire.span('Query analysis') as query_span:
                logfire.info('Analyzing user request', request=user_request)
                query_result = await query_analyzer.run(user_request, deps=deps)
                query_span.set_attribute('generated_query', query_result.data.query_text)
            
            # Search execution phase
            with logfire.span('Search execution') as search_span:
                logfire.info('Executing search query', query=query_result.data.query_text)
                query_results = search_weaviate_collection(
                    weaviate_client,
                    courses_collection,
                    query=query_result.data.query_text
                )
                search_span.set_attribute('results_count', len(query_results))

            # Results summarization phase
            with logfire.span('Results summarization') as summary_span:
                logfire.info('Summarizing search results')
                result = await results_summarizer.run(query_results, deps=deps)
                summary_span.set_attribute('courses_found', len(result.data.relevant_courses))

            logfire.info('Search workflow completed successfully')
            print(result.data)
            
        except Exception as e:
            logfire.error('Search workflow failed', error=str(e))
            workflow_span.record_exception(e)
            raise

nest_asyncio.apply()
await main()

01:53:36.298 Initialize query analyzer agent
01:53:36.300 Initialize results summarizer agent
01:53:36.301 Course search workflow
01:53:36.301   Query analysis
01:53:36.301     Analyzing user request
01:53:36.301     query_analyzer run prompt=Neural Network courses
01:53:36.301       Generate analyzer system prompt
01:53:36.302       preparing model and tools run_step=1
01:53:36.302       model request


Logfire project URL: ]8;id=364213;https://logfire.pydantic.dev/cshorten/weaviate-integration-testing\https://logfire.pydantic.dev/cshorten/weaviate-integration-testing]8;;\

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x15fdbfc40>


01:53:37.634         Pydantic nullable validate_python
01:53:37.635         Pydantic nullable validate_python
01:53:37.636         Pydantic nullable validate_python
01:53:37.637       handle model response
01:53:37.637         Pydantic SearchQuery validate_json
01:53:37.639   Search execution
01:53:37.639     Executing search query
01:53:39.390   Results summarization
01:53:39.391     Summarizing search results
01:53:39.391     results_summarizer run prompt=Search Result: 1:
courseDescription:Deep dive into neural netw...ntlyEnrolling:FalsecourseTitle:Quantum Computing Fundamentals

01:53:39.391       preparing model and tools run_step=1
01:53:39.392       model request
01:53:46.494         Pydantic nullable validate_python
01:53:46.495         Pydantic nullable validate_python
01:53:46.495         Pydantic nullable validate_python
01:53:46.496       handle model response
01:53:46.496         Pydantic SearchResult validate_json
01:53:46.497   Search workflow completed successfully
summ

# 
![](./pydantic-logfire.png)
